In [16]:
import asyncio, time, sys
import websockets, json
import numpy as np
import syncmetrics as syncm
from numpy_ringbuffer import RingBuffer

from pythonosc import osc_message_builder
from pythonosc import osc_bundle_builder
from pythonosc import udp_client
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

In [17]:
# OSC output config
local_ip = '127.0.0.1'
local_port = 12000
output_address = '/results'
client = udp_client.SimpleUDPClient(local_ip, local_port)

#    Send to Processing/MAX/Etc...
def msg(values):
    print(values[0])
    client.send_message("/results", values[0])

#    Send to Processing/MAX/Etc...
def send_osc_bundle(values):
    bundle = osc_bundle_builder.OscBundleBuilder(osc_bundle_builder.IMMEDIATELY)
    msg = osc_message_builder.OscMessageBuilder(address=output_address)
    for arg_to_add in values:
        msg.add_arg(arg_to_add)
    bundle.add_content(msg.build())
    bundle = bundle.build()
    client.send(bundle)

def output_labeled(json_data):
    print(json_data)
    for key, value in json_data:
        output_address = "/result/" + label
        msg = osc_message_builder.OscMessageBuilder(
            address=output_address)
        arg_to_add = value
        msg.add_arg(arg_to_add)
        msg.build()
        self.client.send(msg)
#     bundle = osc_bundle_builder.OscBundleBuilder(
#         osc_bundle_builder.IMMEDIATELY)
    

In [18]:
class Session:
    device_data = json.dumps({})
    uri = 'ws://localhost:8000'
    ws_timeout = 5.0
    num_users = 2

    # to do processing in windoes 
    fs=1000
    buffer_W=150
    win_size=15*fs
    window_A = np.zeros(win_size)
    window_B = np.zeros(win_size)
    no_buffer=int(win_size/buffer_W)
    
    ema_buffers = RingBuffer(capacity=15, dtype=(float, num_users)) 
    feature_buffers = RingBuffer(capacity=no_buffer, dtype=(float, num_users))

    def calculate_features(self):
        device_data = json.loads(self.device_data)
#         self.ema_buffers.appendleft(device_data["CH1"][0], device_data["CH2"][0])
#         sigA = abs(self.ema_buffers[0][0] - self.ema_buffers[0][-1]) / 15
#         sigB = abs(self.ema_buffers[1][0] - abs(self.ema_buffers[1][-1] / 15
#         self.feature_buffers.appendleft([sigA, sigB])

        self.feature_buffers.appendleft([float(device_data["A1"][0]), float(device_data["A2"][0])])
        if self.feature_buffers.shape[0] > self.no_buffer-1:
            buff_A = np.array([i[0] for i in np.array(self.feature_buffers)])
            buff_B = np.array([i[1] for i in np.array(self.feature_buffers)])
            result_1=syncm.lin_reg_r_metric(buff_A, buff_B)
            result_2=syncm.cos_similarity(buff_A, buff_B).flat[0]
            result_3=syncm.correlation_coeff(buff_A, buff_B).flat[0]

            #add results here
            results = [(result_3)]
            print(results)
#             msg(results)

    async def ServerBIT_recieve(self, uri):
        async with websockets.connect(uri) as websocket:
            while True:
                self.device_data = await websocket.recv()
                self.calculate_features()
                await asyncio.sleep(0.0)

    def run(self):
        uri = self.uri
        print("Serving on {}".format(uri))
        try:
            asyncio.get_event_loop().run_until_complete(
                self.ServerBIT_recieve(uri))
        except Exception as e:
            print(e)
            pass
        finally:
            print("no connection, try again")
            exit

In [20]:
session = Session()
session.run()

Serving on ws://localhost:9001
[-0.10911616546888947]
[-0.09101633253122651]
[-0.06635136450164468]
[-0.06152972950593818]
[-0.05774341829902348]
[-0.060177070007654036]
[-0.05502689128518665]
[-0.09448051011409225]
[-0.0831336472800868]
[-0.08453108354118966]
[-0.07551405870215983]
[-0.0896091098691568]
[-0.07513387229576592]
[-0.08317965874469953]
[-0.0938582699166551]
[-0.09500787597694631]
[-0.10976493434809691]
[-0.11143562191389432]
[-0.13581303274626869]
[-0.1392122480969626]
[-0.1333415365358281]
[-0.1298474510034751]
[-0.18529026996767478]
[-0.17846444708319537]
[-0.19948839957535056]
[-0.1994414866441022]
[-0.19984355404227452]
[-0.21775516438021625]
[-0.21574246349263718]
[-0.19701095740606267]
[-0.1789930604368388]
[-0.17757012247443943]
[-0.17700831373718176]
[-0.19098619745537035]
[-0.20837765516324358]
[-0.1808332021632992]
[-0.18096851640586595]
[-0.1811988780572045]
[-0.19400106342931525]
[-0.20403806104273692]
[-0.20573511947033932]
[-0.19763999414176375]
[-0.20765271

[-0.299745309752491]
[-0.29404516133259584]
[-0.2859742231473076]
[-0.280603952071687]
[-0.2696661149273261]
[-0.256421901482116]
[-0.24403281824706646]
[-0.2442543301509229]
[-0.24267940941767066]
[-0.2566591698410617]
[-0.2824315159452383]
[-0.2660990741271477]
[-0.27097560732789727]
[-0.2672310932900924]
[-0.27242309378626556]
[-0.2546450323266566]
[-0.24485829935265166]
[-0.25393347798483923]
[-0.26493057719167074]
[-0.2735562882490961]
[-0.26702751281507836]
[-0.24169549522063813]
[-0.24561650536091206]
[0.020203955548559575]
[-0.23401794806412074]
[0.028658157912006064]
[-0.22947144832057445]
[-0.2321630451915109]
[-0.22779451195713582]
[-0.18373139440828465]
[-0.17705809643543843]
[-0.17331679354795204]
[-0.16233358215289173]
[-0.1373288004263517]
[-0.13556085770887127]
[-0.13364499262620097]
[-0.10916389458820235]
[-0.1331886936582976]
[-0.12650466708457853]
[-0.14067387872843207]
[-0.1147628174188985]
[-0.07074932324898496]
[-0.07478178165190559]
[-0.062032496159618436]
[-0.05

[-0.17643937937344037]
[-0.1746844783396114]
[-0.1836385056454365]
[-0.1816138160329184]
[-0.19460951402586343]
[-0.19803409755959447]
[-0.19635489487703814]
[-0.19457873138521964]
[-0.2624424916968012]
[-0.17185906677684604]
[-0.27214501170797056]
[-0.1596902792558219]
[-0.27696490050166517]
[-0.14579134578386074]
[-0.2714837393173614]
[-0.13757652557954905]
[-0.30971694803182576]
[-0.12499955803905054]
[-0.3010912961196188]
[-0.12075330630326547]
[-0.3446541686921341]
[-0.11686969858665998]
[-0.10782048719573344]
[-0.1013866825835724]
[-0.1061614477394572]
[-0.098391895382478]
[-0.09729349352382172]
[-0.32871676456034316]
[-0.09218932900698021]
[-0.09736690731831206]
[-0.09850855924336743]
[-0.076292596108434]
[-0.0950345349873843]
[-0.10155091384705356]
[-0.10701137760672313]
[-0.08310140959790731]
[-0.3161640628844228]
[-0.07658388719323025]
[-0.31553810712236]
[-0.10193643965305]
[-0.30422726667362043]
[-0.09840338378045406]
[-0.3000654028139497]
[-0.09016947682799599]
[-0.3038269

[-0.07716173606904352]
[-0.19944163328719983]
[-0.06766655848560803]
[-0.19101878625194346]
[-0.05059456086230462]
[-0.19590727401334238]
[-0.07221123486342776]
[-0.1809891462565276]
[-0.07931749176499536]
[-0.17396214974637447]
[-0.07988405656485273]
[-0.17401482792049267]
[-0.08564638582476673]
[-0.17389094798115873]
[-0.0975751868123795]
[-0.17633486949489593]
[-0.09374460114881651]
[-0.1677912189006661]
[-0.10073324058935006]
[-0.16922747513577788]
[-0.11281133724291811]
[-0.16762866185208466]
[-0.09803032716754924]
[-0.15115587357134264]
[-0.1077785205529034]
[-0.13898599442389165]
[-0.11033124990295633]
[-0.13733127152904998]
[-0.11983736000622013]
[-0.14868284981359822]
[-0.10286296292223898]
[-0.1454579170181615]
[-0.09860185529814201]
[-0.12840599362147115]
[-0.09983211739720084]
[-0.11537267234467051]
[-0.08886290915970707]
[-0.11152758200858266]
[-0.07214571086876562]
[-0.1072126859703548]
[-0.08632898282334601]
[-0.09267058304737237]
[-0.10070387268025728]
[-0.0890033443313

[0.053833101869326404]
[-0.21055312916957403]
[0.025681727619293396]
[-0.20808053133984647]
[-0.19769127237629536]
[-0.18795431378446104]
[-0.0002775054043702894]
[-0.2099768987488562]
[-0.004460208337387251]
[-0.20379007959132112]
[0.003381020724104221]
[-0.19489716007667668]
[-0.019948066769975233]
[-0.16907393989160605]
[-0.16683157293437376]
[-0.13453217407401732]
[-0.12513106169533694]
[-0.005149612642308974]
[-0.12882668856726587]
[-0.0005341988468783252]
[-0.14824653984208097]
[0.008337651018971815]
[-0.14898137895220215]
[0.001815601658249875]
[-0.14201483953315852]
[0.005710449353338963]
[-0.13813026382834742]
[-0.03798684890896933]
[-0.1388259689506384]
[-0.035070757825456114]
[-0.13275056260657075]
[-0.027383491219319506]
[-0.14346071873029234]
[-0.008987508615021579]
[-0.1502271662917575]
[0.008181554554487607]
[-0.1611823947026652]
[-0.009049608324893823]
[-0.15868881525491316]
[-0.173894413874667]
[-0.18184399789421626]
[-0.17913045081585424]
[-0.1831364819680247]
[-0.176

[-0.11391881137465917]
[-0.0757243966820176]
[-0.10385806926997708]
[-0.06550065864441888]
[-0.11157666827915119]
[-0.10638599884308127]
[-0.10479524650756472]
[-0.11407877855785852]
[-0.11762829019326093]
[-0.10811510759096826]
[-0.0990224410575483]
[-0.06120077240262723]
[-0.07191256464568214]
[-0.05655202268731888]
[-0.037836211987018344]
[-0.04161153870831417]
[0.0007949487012455876]
[0.0028698983115737708]
[-0.022248037593597562]
[-0.025089374336324025]
[-0.034780959030725554]
[-0.04267755879920086]
[-0.04215731896151703]
[0.009847473567343059]
[0.00760955619413804]
[-0.0005894422597110198]
[0.022177307489819015]
[0.032697880592837625]
[0.041149112213418444]
[0.04091220286852706]
[0.0633493231155503]
[0.08917172505602565]
[0.08874787705612132]
[0.04694524878435582]
[0.044662787058086774]
[0.03371434246397341]
[0.03365250254938313]
[-0.09497905620101696]
[0.03577230816861901]
[-0.06545712888901999]
[0.013475244484190728]
[-0.0786886522413294]
[-0.019249646944468644]
[-0.07585803425

[-0.27892602608290573]
[-0.2681564885784242]
[-0.2670944663315311]
[-0.256953021717971]
[-0.25798590935921106]
[-0.2649443504321617]
[-0.26985536459978204]
[-0.24652944850715067]
[-0.23972499215234588]
[-0.25771272140488277]
[-0.252028307302367]
[-0.2490986222995452]
[-0.23741655621535634]
[-0.23268900000613885]
[-0.2331299735841903]
[-0.22985135991347644]
[-0.22672736750262404]
[-0.20201476248444591]
[-0.2009197275973534]
[-0.20222858810163008]
[-0.18399819434907075]
[-0.18943847161050206]
[-0.18442874077416768]
[-0.18437584944755672]
[-0.1806669750534591]
[-0.16184785550516056]
[-0.15299705935342203]
[-0.13873239422349581]
[-0.12442256254578009]
[-0.12153157922557832]
[-0.11271033399550062]
[-0.11610645849015606]
[-0.0862695513841888]
[-0.09384342055629175]
[-0.07929653349058885]
[-0.052003851559190566]
[-0.026579385718980927]
[-0.040220917082403274]
[-0.03647453317590994]
[-0.019540418731210884]
[-0.0168402635368132]
[-0.02147475052037267]
[-0.027839407865791175]
[-0.029149807690404

[-0.04136482581419975]
[0.07434557153354505]
[-0.05384148936529863]
[0.09861809432063004]
[-0.06570015078479571]
[-0.0518387897335809]
[-0.04604671096559067]
[0.08361863261927545]
[-0.051429991299757075]
[0.08200231595834771]
[-0.05495263381308082]
[-0.07012466061717776]
[-0.06070073026764557]
[-0.06033812630262173]
[-0.05733367137970161]
[-0.06564360371035496]
[-0.061172311389701986]
[0.07765286895201234]
[-0.08503015790715263]
[0.07304844614685727]
[-0.07378920159608043]
[0.0813930654613354]
[-0.08174768885732576]
[0.08077048988719744]
[-0.07566796743528527]
[0.06861807552452831]
[-0.0821533182697489]
[0.07012422120488528]
[-0.09553377287188791]
[0.04455653262811106]
[-0.10743033238580792]
[0.06028868888298486]
[-0.0944516971819844]
[0.05933415821628151]
[-0.09516640202672635]
[0.07468750517737013]
[-0.10569284012618568]
[0.07598086335847178]
[-0.09460582063194058]
[0.0847172315041414]
[-0.1297214506809039]
[0.06643537089533752]
[-0.13892244747090465]
[0.07934578165428317]
[-0.122718

[-0.17895092944525517]
[-0.1751849190374207]
[-0.18700852061196435]
[-0.18345179342764656]
[-0.20680086093114028]
[-0.20629097341623034]
[-0.20793622008809534]
[-0.22455594181917551]
[-0.23354520637902682]
[-0.24189782367976934]
[-0.24470005204567016]
[-0.24865445634929947]
[-0.27135842162814383]
[-0.2558535433886323]
[-0.23991279981902186]
[0.1052271564324135]
[-0.24875160529375628]
[0.12922977601437322]
[-0.22527960462150873]
[0.15214083751569044]
[-0.2131391396866873]
[0.13618306525107193]
[-0.2161835159325474]
[0.1458431411020851]
[-0.18500754267371983]
[-0.18398301713707554]
[-0.20918788543218503]
[-0.2238087285282927]
[-0.24308054471345011]
[-0.26175796757905584]
[-0.28080657499158546]
[-0.2919986190501263]
[-0.2732638481218904]
[-0.2730842150281431]
[-0.2816383652203136]
[-0.29363610224162284]
[-0.3037840545692534]
[-0.28042355786119183]
[-0.2946018170136339]
[-0.29538691836899167]
[-0.3098742377570279]
[-0.3048681325881639]
[0.1264360495475323]
[-0.30711650083097286]
[-0.325959